# Cellucid Jupyter Integration Tutorial

This notebook demonstrates how to use **cellucid** directly within Jupyter notebooks for interactive 3D visualization of single-cell data.

## Overview

Cellucid provides seamless Jupyter integration that allows you to:

1. **Visualize datasets** - Embed the full 3D cellucid viewer directly in notebook cells
2. **Interact programmatically** - Highlight cells, change coloring, and more from Python
3. **Work with remote data** - View data on remote servers via SSH tunnels

The visualization connects to the same powerful WebGL-based viewer at [cellucid.com](https://www.cellucid.com), but serves data from your local machine.

## Prerequisites

Make sure you have cellucid installed with Jupyter support:

```bash
pip install cellucid[jupyter]
```

Or for development:
```bash
pip install -e ".[jupyter]"
```

In [ ]:
import sys
from pathlib import Path

HERE = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()

def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "pyproject.toml").exists():
            return candidate
    return start


PROJECT_ROOT = find_project_root(HERE)
SRC_DIR = PROJECT_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

## 1. Quick Start - The `show()` Function

The simplest way to visualize a cellucid dataset is using the `show()` function.
Just point it to a directory containing exported cellucid data.

In [ ]:
from cellucid import show

# Path to the suo dataset (Human Developmental Cell Atlas data)
# This dataset contains ~500K cells with cell type annotations
DATA_PATH = "/Users/kemalinecik/git_nosync/_/cellucid/assets/exports/suo"

In [ ]:
# Display the viewer - this will:
# 1. Start a local data server
# 2. Embed the cellucid web viewer in an iframe
# 3. The viewer loads data from your local server

viewer = show(DATA_PATH, height=700)

### What just happened?

Behind the scenes, cellucid:

1. **Started a local HTTP server** on an available port (default: 8765)
2. **Embedded the viewer** from cellucid.com in an iframe
3. **Connected the viewer** to your local data via the `?remote=` URL parameter

The viewer is now displaying your dataset with full interactivity:
- **Rotate/zoom** with mouse
- **Click cells** to see their metadata
- **Use the control panel** (right side) to change coloring, filtering, etc.

## 2. The CellucidViewer Class

For more control, use the `CellucidViewer` class directly.
This gives you access to programmatic interaction methods.

In [ ]:
from cellucid import CellucidViewer

# Create a viewer with specific settings
viewer = CellucidViewer(
    data_dir=DATA_PATH,
    port=8766,           # Specific port (optional, auto-selected if not specified)
    height=300,          # Height in pixels
    interactive=True,    # Enable bidirectional communication
    auto_open=False,     # Don't display immediately
)

print(f"Server URL: {viewer.server_url}")
print(f"Viewer URL: {viewer.viewer_url}")

In [ ]:
# Now display the viewer
viewer.display()

## 3. Programmatic Interaction

One of the most powerful features is the ability to control the viewer from Python code.
This enables interactive analysis workflows.

### 3.1 Highlighting Cells

You can highlight specific cells to draw attention to them.
This is useful when you identify interesting cells through analysis.

In [ ]:
# Highlight the first 100 cells in red
viewer.highlight_cells(list(range(10000)), color="#ff0000")

In [ ]:
# Highlight cells 500-600 in blue
viewer.highlight_cells(list(range(5000, 6000)), color="#0066ff")

In [ ]:
# Clear all highlights
viewer.clear_highlights()

### 3.2 Changing the Color Scheme

You can programmatically change which field is used for coloring the points.

In [ ]:
# Color by a categorical field (e.g., cell type)
# The exact field name depends on your dataset
viewer.set_color_by("cell_type")

In [ ]:
# Color by a continuous field (e.g., gene expression)
# viewer.set_color_by("total_counts")  # Uncomment if this field exists

### 3.3 Controlling Cell Visibility

You can show/hide specific cells to focus on subsets of your data.

In [ ]:
# Hide cells 0-1000
viewer.set_visibility(list(range(1000)), visible=False)

In [ ]:
# Show them again
viewer.set_visibility(list(range(1000)), visible=True)

### 3.4 Camera Control

In [ ]:
# Reset camera to default view
viewer.reset_view()

## 4. Integration with Analysis Workflows

The real power of Jupyter integration comes from combining visualization with analysis.
Here's an example workflow.

In [ ]:
import numpy as np

# Example: Identify cells of interest through analysis
# (In practice, this would come from your actual analysis)

# Simulate finding interesting cells (e.g., highly expressed gene)
np.random.seed(42)
interesting_cells = np.random.choice(10000, size=200, replace=False).tolist()

print(f"Found {len(interesting_cells)} interesting cells")

In [ ]:
# Highlight these cells in the viewer
viewer.highlight_cells(interesting_cells, color="#00ff88")

## 5. Working with Remote Servers (SSH Tunnels)

When your data is on a remote server, you can still use cellucid:

### On the remote server:
```bash
# Start the cellucid server
cellucid /path/to/data --port 8765
```

### On your local machine:
```bash
# Create an SSH tunnel
ssh -L 8765:localhost:8765 user@remote-server
```

### Then in your notebook:
```python
# Point to the tunneled port
viewer = CellucidViewer("http://localhost:8765")
```

This way, the data never leaves the server - only visualization commands and minimal metadata are transferred.

## 6. Cleanup

When you're done, stop the server to free up resources.

In [ ]:
# Stop the viewer and its data server
viewer.stop()

print("Viewer stopped")

## Summary

Key takeaways:

| Feature | Function |
|---------|----------|
| Quick view | `show("/path/to/data")` |
| Full control | `CellucidViewer("/path/to/data")` |
| Highlight cells | `viewer.highlight_cells([1, 2, 3])` |
| Change coloring | `viewer.set_color_by("field_name")` |
| Hide/show cells | `viewer.set_visibility([...], visible=False)` |
| Reset view | `viewer.reset_view()` |
| Cleanup | `viewer.stop()` |

### Next Steps

- Explore the [cellucid documentation](https://cellucid.readthedocs.io) for more features
- Try the web viewer directly at [cellucid.com](https://www.cellucid.com)
- Export your own data using `cellucid.export_data_for_web()`